# BenzDB tools

## Source code

### Module imports

In [1]:
import ipywidgets as w
import requests
import json
import base64 
import io
from PIL import Image
import IPython as ip
import jupyter_jsmol as Jsmol
import pandas as pd
import matplotlib.pyplot as plt

### Definition of criterion classes

#### Definition of the main criterion class

In [2]:
class Criterion:
    """ This class allows for representing criterions """
    
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        self.__key = key
        self.__description  = description     
    
    
    def display (self):
        """ displays the widget corresponding to the criterion """
        pass
    
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        pass
    
    
    def get_description (self) -> str:
        """ returns the description """
        return self.__description
    
    def get_key (self) -> str:
        """ returns the key related to the criterion """
        return self.__key

#### Definition of criterion child classes

In [3]:
class Int_Criterion (Criterion):
    """ This class allows for representing criterions based on int value """
 
    def __init__ (self, key: str, description: str, initial_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(value=1, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [4]:
class Float_Criterion (Criterion):
    """ This class allows for representing criterions based on float value """
 
    def __init__ (self, key: str, description: str, initial_value: float):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [5]:
class String_Criterion (Criterion):
    """ This class allows for representing criterions based on string value """
 
    def __init__ (self, key: str, description: str, initial_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(value=1, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [6]:
class Query_Criterion (Criterion):
    """ This class allows for representing the desired query """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(description=self.get_description(), options=["benzenoids","ir","ims2d1a","nics","clar_covers","properties","irregularities"], value='benzenoids', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        return str(self.__element.value)

### Definition of the form class

In [7]:
class Form:
    
    def __init__ (self):
        """ initialise the form """
        self.__query = None         # the query
        self.__demand_type = None   # the type of demand
        self.__json_string = None   # the query as a JSON string
        self.__data = None          # the data related to the query (if the query succeeds)
        
        self.create_form()
        
        
    def create_form(self) -> None:
        """ creates the form allowing for choosing the values of desired criteria """
        self.__criteria = []

        # criteria about basic information
        self.__criteria.append (Int_Criterion(key="nbHexagons", description="# hexagons", initial_value=1))
        self.__criteria.append (Int_Criterion(key="nbCarbons", description="# carbons", initial_value=1))
        self.__criteria.append (Int_Criterion(key="nbHydrogens", description="# hydrogens", initial_value=1))

        # query
        self.__criteria.append (Query_Criterion(key="query", description="query"))    

        # validation buttons
        count_btn = w.Button (description="Count")
        count_btn.on_click (self.perform_query)

        getdata_btn = w.Button (description="Get Data")
        getdata_btn.on_click (self.perform_query)

        getquery_btn = w.Button (description="Get JSON Query")
        getquery_btn.on_click (self.perform_query)
        
        box = w.HBox([count_btn, getdata_btn,getquery_btn])
        display(box)
    
    
    def perform_query (self, btn) -> None:
        """ performs the query and sets the corresponding attributes """ 
        # we identify the type of demand
        if btn.description == "Count":
            self.__demand_type = "count"
        elif btn.description == "Get Data":
            self.__demand_type = "data"
        elif btn.description == "Get JSON Query":
            self.__demand_type = "json"
        else:
            self.__demand_type = "unknown"

        # we build the JSON string and the corres
        self.__json_string = "{\n"
        for c in self.__criteria:
            if isinstance(c,Query_Criterion):
                self.__query = c.get_criterion()
            else:
                s = c.get_criterion()
                if len(s) > 0:
                    if len(self.__json_string) > 3:
                        self.__json_string += ",\n"
                    self.__json_string += "\t" + s
        self.__json_string += "\n}"

        if self.__demand_type == "data":
            response = requests.post("https://benzenoids.lis-lab.fr/find_"+self.__query, json= json.loads(self.__json_string))
        elif self.__demand_type == "count":
            response = requests.post("https://benzenoids.lis-lab.fr/count_"+self.__query, json= json.loads(self.__json_string))
        
        if self.__demand_type in ["data","count"]:
            if response.status_code == 200:
                self.__data = response.json()

        
    def get_data (self):
        """ returns the data related to the query """
        return self.__data
    
    
    def get_query (self):
        """ returns the desired query """
        return self.__query

    
    def get_json_string (self):
        """ returns the JSON string related to the desired query """
        return self.__json_string
    

    def get_demand_type (self):
        """ returns the type of the current demand """
        return self.__demand_type

### Definition of the display classes

In [8]:
class Display:
    """ This class allows for displaying benzenoid information """
    
    def __init__ (self, info: dict):
        """ initializes the display tool """
        self.__information = info
        self.__data = {}
        self.add_data ("Benzenoid id", self.get_information("idBenzenoid"))
        self.add_data ("InChI", self.get_information("inchi"))
        self.add_data ("Label", self.get_information("label"))
        self.add_data ("#hexagons", self.get_information("nbHexagons"))
        self.add_data ("#carbons", self.get_information("nbCarbons"))
        self.add_data ("#hydrogens", self.get_information("nbHydrogens"))
        self.add_data ("Weight", self.get_information("weight"))
        self.add_data ("Irregularity", self.get_information("irregularity"))
    
    
    def add_data (self, label, value) -> None:
        """ adds a label to display with its corresponding value """
        self.__data [label] = [value]
        
    
    def display (self) -> None:
        """ displays the information """           
        self.__df = pd.DataFrame(self.__data)
        self.__df.index = [""]
        
        ip.display.display(self.__df.transpose())
        self.display_molecule()
        
    
    def display_molecule (self) -> None:
        """ displays the molecule thanks to Jsmol """
        view = Jsmol.JsmolView.from_str(str(self.get_information("nbCarbons")+self.get_information("nbHydrogens"))+"\nComment\n"+self.get_information("geometry"))
        ip.display.display(view)
        
    
    def display_image (self, str64: str) -> None:
        """ displays the base-64 image defined by str64 """
        img = Image.open(io.BytesIO(base64.b64decode(str64)))
        ip.display.display(img)
        
    
    def get_information (self, key) -> dict:
        """ returns the information """
        return self.__information[key]

In [9]:
class Display_IR (Display):
    """ This class allows for displaying benzenoid information from IR query """

    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("Final energy", self.get_information("finalEnergy"))
        
    
    def display (self) -> None:
        """ displays the information """
        super().display()
               
        x = [float(v) for v in self.get_information("frequencies").split(" ")]
        y = [float(v) for v in self.get_information("intensities").split(" ")]
        
        plt.cla()
        plt.plot(x, y)
        plt.xlabel('frequencies')
        plt.ylabel('intensities')
        plt.show()


In [10]:
class Display_IMS2D1A (Display):
    """ This class allows for displaying benzenoid information from IMS2D1A query """

    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("Type", self.get_information("type"))

    def display (self) -> None:
        """ displays the information """
        super().display()
        self.display_image(self.get_information("picture"))

In [11]:
class Display_NICS (Display):
    """ This class allows for displaying benzenoid information from NICS query """
    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("NICS values", self.get_information("nics"))

In [12]:
class Display_Clar_Covers (Display):
    """ This class allows for displaying benzenoid information from Clar Cover query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        self.display_image(self.get_information("clarCover"))

In [13]:
class Display_Properties (Display):
    """ This class allows for displaying benzenoid information from properties query """

    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        yesno = ["no","yes"]
        self.add_data ("Catacondensed", yesno[self.get_information("catacondensed")])
        self.add_data ("Coronoid", yesno[self.get_information("coronoid")])
        self.add_data ("Coronenoid", yesno[self.get_information("coronenoid")])
        self.add_data ("Symmetry", self.get_information("symmetry"))
        self.add_data ("#Kekulé structures", self.get_information("kekule"))
        self.add_data ("HOMO", self.get_information("homo"))
        self.add_data ("LUMO", self.get_information("lumo"))
        self.add_data ("Dipole moment", self.get_information("moment"))
    
    def display (self) -> None:
        """ displays the information """
        super().display()


In [14]:
class Display_Irregularities (Display):
    """ This class allows for displaying benzenoid information from irregularities query """
    
    def __init__ (self, info: dict):
        """ initializes the display tool """
        super().__init__(info)
        self.add_data ("# solo", self.get_information("solo"))
        self.add_data ("# duo", self.get_information("duo"))
        self.add_data ("# trio", self.get_information("trio"))
        self.add_data ("# quartet", self.get_information("quartet"))
        

## Filling the form

In [15]:
form = Form()


## Processing the result molecule per molecule

In [24]:
demand_type = form.get_demand_type()

if demand_type == "data":
    data = form.get_data()
    query = form.get_query()
    
    for molecule in data:
        if query == "benzenoids":
            d = Display(molecule)
        elif query == "ir":
            d = Display_IR(molecule)
        elif query == "ims2d1a":
            d = Display_IMS2D1A(molecule)
        elif query == "nics":
            d = Display_NICS(molecule)
        elif query == "clar_covers":
            d = Display_Clar_Covers(molecule)
        elif query == "properties":
            d = Display_Properties(molecule)
        elif query == "irregularities":
            d = Display_Irregularities(molecule)
        d.display()
        
elif demand_type == "count":
    print ("Number of molecules:",form.get_data())
    
elif demand_type == "json":
    print ("JSON query:")
    print (form.get_json_string())

,
Benzenoid id,3532
InChI,1S/C14H10/c1-3-7-13-11(5-1)9-10-12-6-2-4-8-14(...
Label,1-3-4
#hexagons,3
#carbons,14
#hydrogens,10
Weight,178.22976
Irregularity,0.8
# solo,0
# duo,2


JsmolView(layout=Layout(align_self='stretch', height='400px'))

,
Benzenoid id,4421
InChI,1S/C14H10/c1-2-6-12-10-14-8-4-3-7-13(14)9-11(1...
Label,0-4-8
#hexagons,3
#carbons,14
#hydrogens,10
Weight,178.22976
Irregularity,0.8
# solo,2
# duo,0


JsmolView(layout=Layout(align_self='stretch', height='400px'))

,
Benzenoid id,4674
InChI,1S/C13H22/c1-4-10-6-2-8-12-9-3-7-11(5-1)13(10)...
Label,0-3-4
#hexagons,3
#carbons,13
#hydrogens,9
Weight,165.21108
Irregularity,1.0
# solo,0
# duo,0


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [17]:
import pandas as pd


df = pd.DataFrame({
    "strings": ["Adam", "Mike"],
    "ints": [1, 3],
    "floats": [1.123, 1000.23]
})

df 

df.style.relabel_index(["row 1", "row 2"], axis=0)
  #.format(precision=5, thousands=",", decimal=".") \
  #.format_index(str.upper, axis=1) \
df.index = ["", ""]

df.transpose()

,,
strings,Adam,Mike
ints,1,3
floats,1.123,1000.23
